In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os
import cv2
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model
from keras.initializers import glorot_uniform
import tensorflow as tf
from sklearn.metrics import confusion_matrix, classification_report
import json
import random

In [6]:
# Dataset
dataset_path = 'Bacteria_Dataset_unzipped'

In [7]:
bacteria_types = os.listdir(dataset_path)
print("Types of bacteria found: ", len(bacteria_types))

Types of bacteria found:  19


In [8]:
# Loading the data
bacteria = []
for item in bacteria_types:
    all_bacteria = os.listdir(os.path.join(dataset_path, item))
    for img_name in all_bacteria:
        img_path = os.path.join(dataset_path, item, img_name)
        bacteria.append((item, img_path))
# print(bacteria)

In [9]:
# Create a dataframe
bacteria_df = pd.DataFrame(data=bacteria, columns=['bacteria type', 'image'])
bacteria_df

,bacteria type,image
0,Acinetobacter.baumanii,Bacteria_Dataset_unzipped\Acinetobacter.bauman...
1,Acinetobacter.baumanii,Bacteria_Dataset_unzipped\Acinetobacter.bauman...
2,Acinetobacter.baumanii,Bacteria_Dataset_unzipped\Acinetobacter.bauman...
3,Acinetobacter.baumanii,Bacteria_Dataset_unzipped\Acinetobacter.bauman...
4,Acinetobacter.baumanii,Bacteria_Dataset_unzipped\Acinetobacter.bauman...
...,...,...
404,Veionella,Bacteria_Dataset_unzipped\Veionella\Veionella_...
405,Veionella,Bacteria_Dataset_unzipped\Veionella\Veionella_...
406,Veionella,Bacteria_Dataset_unzipped\Veionella\Veionella_...
407,Veionella,Bacteria_Dataset_unzipped\Veionella\Veionella_...


In [10]:
print("Total number of images in the dataset: ", len(bacteria_df))

Total number of images in the dataset:  409


In [11]:
# Label encoder
y = bacteria_df['bacteria type'].values
y_labelencoder = LabelEncoder()
y = y_labelencoder.fit_transform(y)
y = y.reshape(-1, 1)
onehotencoder = OneHotEncoder(sparse_output=False)
Y = onehotencoder.fit_transform(y)

In [13]:
#preprocess the images
im_size = 64
images = []
for img_path in bacteria_df['image']:
    img = cv2.imread(img_path)
    img = cv2.resize(img, (im_size, im_size))
    images.append(img)

In [14]:
images = np.array(images, dtype='float32') / 255.0


In [15]:
# Split your dataset into training and testing
images, Y = shuffle(images, Y, random_state=1)
train_x, test_x, train_y, test_y = train_test_split(images, Y, test_size=0.2, random_state=42)


# Identity Block

In [16]:
def identity_block(X, f, filters, stage, block):
    """
   
    Arguments:
    X -- input of shape (m, height, width, channel)
    f -- shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    
    Returns:
    X -- output of the identity block, tensor of shape (n_H, n_W, n_C)
    """
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Saving the input value 
    X_shortcut = X
    
    # First component of main path
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2a')(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    
    # Second component of main path
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b')(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path 
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c')(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation 
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    return X

# Convolutional Block

In [17]:

def convolutional_block(X, f, filters, stage, block, s = 2):
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_shortcut = X


    # First layer 
    X = Conv2D(F1, (1, 1), strides = (s,s), name = conv_name_base + '2a')(X) # 1,1 is filter size
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)  # normalization on channels
    X = Activation('relu')(X)

      
    # Second layer  (f,f)=3*3 filter by default
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b')(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)


    # Third layer
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c')(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)


    ##### SHORTCUT PATH #### 
    X_shortcut = Conv2D(filters = F3, kernel_size = (1, 1), strides = (s,s), padding = 'valid', name = conv_name_base + '1')(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3, name = bn_name_base + '1')(X_shortcut)

    # Final step: Add shortcut value here, and pass it through a RELU activation 
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    return X


# Creating ResNet 50 

In [18]:
#Each ResNet block is either 2 layer deep
def ResNet50(input_shape=(64, 64, 3), classes=3):
    """
    Implementation of the ResNet50 architecture:
    CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*3
    -> CONVBLOCK -> IDBLOCK*5 -> CONVBLOCK -> IDBLOCK*2 -> AVGPOOL -> TOPLAYER

    """

    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input) #3,3 padding

    # Stage 1
    X = Conv2D(64, (7, 7), strides=(2, 2), name='conv1')(X) #64 filters of 7*7 
    X = BatchNormalization(axis=3, name='bn_conv1')(X) #batchnorm applied on channels
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X) #window size is 3*3

    # Stage 2
    X = convolutional_block(X, f=3, filters=[64, 64, 256], stage=2, block='a', s=1)
    # convolutional_block is a function defined above. Convolutional_block have 3 layers.
    #filters=[64, 64, 256] first 64 is for 1st layer and 2nd 64 is for 2nd layer and 256 is for 3rd layer of convultional block   
    # below are the conv layers from convolutional_block function
    #X = Conv2D(F1, (1, 1), strides = (s,s), name = conv_name_base + '2a')(X)
    #X = Conv2D(F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b')(X)
    #X = Conv2D(F3, (1, 1), strides = (s,s), name = conv_name_base + '2a')(X)
   
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b') 
    #X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2a')(X)
    #X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b')(X)
    #X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c')(X)
  
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')
    #X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2a')(X)
    #X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b')(X)
    #X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c')(X)

    # Stage 3 
    X = convolutional_block(X, f = 3, filters = [128, 128, 512], stage = 3, block='a', s = 2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')

    # Stage 4 
    X = convolutional_block(X, f = 3, filters = [256, 256, 1024], stage = 4, block='a', s = 2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')

    # Stage 5 
    X = convolutional_block(X, f = 3, filters = [512, 512, 2048], stage = 5, block='a', s = 2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

    # AVGPOOL 
    X = AveragePooling2D((2,2), name="avg_pool")(X)

    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
    
    # Create model
    model = Model(inputs = X_input, outputs = X, name='ResNet50')

    return model


In [19]:
# Create and compile the model
model = ResNet50(input_shape=(64, 64, 3), classes=19)

In [20]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [1]:
# model.summary()

In [22]:
# Train your model with validation data
history = model.fit(train_x, train_y, validation_data=(test_x, test_y), epochs=50, batch_size=32)

Epoch 1/50
11/11 [==============================] - 37s 2s/step - loss: 4.2813 - accuracy: 0.1009 - val_loss: 2.9838 - val_accuracy: 0.0244
Epoch 2/50
11/11 [==============================] - 18s 2s/step - loss: 3.7897 - accuracy: 0.2202 - val_loss: 3.0744 - val_accuracy: 0.0610
Epoch 3/50
11/11 [==============================] - 18s 2s/step - loss: 2.3542 - accuracy: 0.3364 - val_loss: 2.9696 - val_accuracy: 0.1341
Epoch 4/50
11/11 [==============================] - 18s 2s/step - loss: 1.7768 - accuracy: 0.4067 - val_loss: 3.0465 - val_accuracy: 0.0610
Epoch 5/50
11/11 [==============================] - 18s 2s/step - loss: 1.6773 - accuracy: 0.5260 - val_loss: 3.1685 - val_accuracy: 0.0610
Epoch 6/50
11/11 [==============================] - 18s 2s/step - loss: 1.2511 - accuracy: 0.5933 - val_loss: 3.5029 - val_accuracy: 0.0732
Epoch 7/50
11/11 [==============================] - 18s 2s/step - loss: 1.7888 - accuracy: 0.5841 - val_loss: 3.9363 - val_accuracy: 0.0610
Epoch 8/50
11/11 [==

In [23]:
# Save the model
model.save('Models/ResNet50_scratch.h5')

C:\Users\Machine Learning GPU\anaconda3\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [24]:
# Save the training history (loss and accuracy) as JSON
history_dict = {
    'loss': history.history['loss'],
    'accuracy': history.history['accuracy'],
    'val_loss': history.history['val_loss'],
    'val_accuracy': history.history['val_accuracy']
}

with open('Results/ResNet50_scratch.json', 'w') as json_file:
    json.dump(history_dict, json_file)